# Projekt MAGN — Implementacja biblioteki do budowy Multi-Associative Graph Network (MAGN) oraz uczenia (klasyfikacja + regresja)
### Autorzy:

- Szymon Jurecki
- Dominik Breksa

### Opis projektu:

Z uwagi na to, że ilość wyprodukowanego przez nas kodu jest zbyt duża, aby umieścić go w jednym pliku, postanowiliśmy stworzyć bibliotekę `magn` w języku python, która realizuje odpowiednie implementacje np. uczenia, tworzenia grafów ASA.

Ponieważ jest to dość rozległy projekt, to pozwolę sobie opisać strukturę projektu:
- Folder `docs` - zawiera wewnętrzną dokumentację, obrazki i zapisy literatury.
- Folder `resources/data` - tutaj znajduje się baza danych później pobrana z platformy Kaggle.
- Folder `src` - ma kod źródłowy paczek python, które stworzyliśmy:
    - Folder `magn` - zawiera paczkę użytą dalej do stworzenia MAGN.
- Folder `tests` - zawiera tymczasowe testy, zbudowanych przez nas method, funkcji i obiektów.
- Plik `.gitignore` - samo opisowe.
- Plik `pyproject.toml` - informacje o paczce niezbędne do instalacji.
- Plik `README.md` - ten tekst.
- PLik `requiements.txt` - zawiera niezbędne paczki zależności.
- Plik `setup.py` - instaluje naszą paczkę.

Następnie pobierzemy bazę danych z platformy Kaggle ([LINK](https://www.kaggle.com)) w formacie SQLite, na jej podstawie zbudujemy konkretny MAGN, potem przeprowadzimy uczenie i przeanalizujemy wyniki w tym notebooku.

### Cel projektu:

1. Implementacja następujących struktur danych dla małej biblioteki:
    - Aggregative Sorting Associative Graphs (ASA-Graphs)
    - Multi-Associative Graph Network (MAGN)
    - Baza danych
2. Przeprowadzenie uczenia na przykładowej bazie danych:
    - Uczenie
    - Predykcja
    - Regresja
    - Obliczenie metryk i porównanie działania.

### Wstęp teoretyczny

Relacyjne bazy danych, pomimo tego, jak często są używane, nie są zbyt dobre w reprezentacji relacji ściśle związanych z podobieństwem między np.: wartościami, kolejnością poszczególnych danych wewnątrz tabel.

W związku z tym wprowadzimy tak zwane Multi-Associative Graph Network (MAGN), zbudowane na podstawie konkretnych baz danych i pokażemy, że można je wykorzystać np. predykcji i regresji.

##### Co to są Aggregative Sorting Associative Graphs (ASA-Graphs)?

W skrócie są to B-Drzewa, gdzie wszystkie wierzchołki są ze sobą połączone w posortowanej kolejności przy pomocy listy dwukierunkowej. Te B-Drzewa mają kilka ograniczeń np. to, że w danym wierzchołku nie mogą być więcej niż 2 elementy.

![ASA template](docs/examples/data/templatepng.png)

Gdzie:
- `Table` - Nasza konwencja podpisywania, z jakiej tabeli jest graf.
- `Column` - Nasza konwencja podpisywania, z jakiej kolumny jest graf.
- `Counter` - Licznik ile razy w danej porcji danych nastąpiło powtórzeń danego obiektu.
- `Key` - Sama wartość elementu.
- `Left Pointer` - Wartość pointera dla lewego krańca listy dwukierunkowej.
- `Right Pointer` - Wartość pointera dla prawego krańca listy dwukierunkowej.
- `pk:Numer` - Numer klucza, jakiego obiekt się tyczy (dla ułatwienia).
- Warto wspomnieć, że na tym zdjęciu jeszcze nie ma wag.

Przykładowy ASA może wyglądać na przykład tak.

![ASA przykładowy](docs/examples/data/animals_species_4.png)

##### Jak stworzyć MAGN?

Należy wykorzystać tak zwany algorytm transformacji asocjatywnej na konkretnej bazie danych. Poniżej przedstawimy pseudokod.

```python
def create_magn():
    db = obtain_database()  # (1)
    
    magn = create_empty_magn()  # (2)
    
    for table in db.get_tables_pk_first():  # (3)
        for column in table:  # (4)
            data = column.data
            asa = create_asa(data)  # (5)
            magn.add_asa(asa, asa.connected_objects)  # (6)
    
    return magn
```

1. Dostań gotową bazę danych wypełnioną kluczami i danymi.
2. Stwórz pusty MAGN, aby móc potem dodać do niego poszczególne ASA.
3. Należy w odpowiedni sposób dostarczać do niego grafy asa, tak, any najpierw dać klucze główne, a potem odsłonione przez nie klucze obce, więc należy dodać je w odpowiedni sposób (konkretnie my wykorzystaliśmy sortowanie topologi-cze z zależności między kluczami).
4. Prze-iteruj przez wszystkie kolumny.
5. Stwórz z nich odpowiedni graf asa.
6. Dodaj go łącznie z przyległymi obiektami do głównego MAGN.

Należy pamiętać, że pomiędzy obiektami i elementami grafu ASA należy wyliczać odpowiednie wagi i połączyć wszystko w całość.

Obliczanie wag:
1. ....
2. ....
3. ....
4. ....

##### Przykładowy MAGN:

Dane i schematy użyte do konstrukcji (pominięto z uwagi na skalę tego niektóre klucze).

![Magn data](docs/examples/data/data.png)

![Magn schema](docs/examples/data/data_schema.png)

Gotowy MAGN stworzony ręcznie.

![Przykładowy MAGN](docs/examples/data/magn.png)

Infografiki własne.

##### Algorytm uczenia

Poniżej zamieszczę pseudo kod uczenia się MAGN:

```python
def fit_magn(magn, data, targets):
    pass
```

1. ...

##### Algorytm predykcji

Poniżej zamieszczę pseudo kod predykcji MAGN:

```python
def predict_magn(magn, data):
    pass
```

1. ...

### Wykorzystane technologie i zależności:
- Python: `3.12.0`
- Pandas: `2.2.2`
- Kaggle: `1.6.14`
- SQLite3: `3.46.0`
- Inne zawarte w `requirements.txt`

### Niezbędne instalacje

Aby móc użyć tego notebooka zalecane jest stworzenie odrębnego wirtualnego środowiska Python -a (venv) i zainstalowania niezbędnych zależności. Sama biblioteka zainstaluje się po wykonaniu linijki: `pip install .`, ale dodatkowo, aby ją przedstawić w tym, notebooku musimy jeszcze doinstalować kilka kolejnych modułów.

In [1]:
%pip install .
%pip install scikit-learn
%pip install numpy
%pip install kaggle

Processing c:\users\domin\pycharmprojects\ggsn---magn
  Installing build dependencies: started
  Installing build dependencies: finished with status 'done'
  Getting requirements to build wheel: started
  Getting requirements to build wheel: finished with status 'done'
  Installing backend dependencies: started
  Installing backend dependencies: finished with status 'done'
  Preparing metadata (pyproject.toml): started
  Preparing metadata (pyproject.toml): finished with status 'done'
  Created wheel for magn: filename=magn-1.0.0.0-py3-none-any.whl size=16641 sha256=7befdde90819fd579b187119435b016028a5bbd0d994fbdab9517f05a3424a24
  Stored in directory: c:\users\domin\appdata\local\pip\cache\wheels\02\69\7a\7a321fc99cdf850f17c5dbe96e53ef1e4b07adda9d5bd3d1f7
Successfully built magn
  Attempting uninstall: magn
    Found existing installation: magn 1.0.0.0
    Not uninstalling magn at c:\users\domin\pycharmprojects\ggsn---magn\src, outside environment C:\Users\domin\PycharmProjects\GGSN--

DEPRECATION: Loading egg at c:\users\domin\pycharmprojects\ggsn---magn\.venv\lib\site-packages\magn-1.0.0.0-py3.12.egg is deprecated. pip 24.3 will enforce this behaviour change. A possible replacement is to use pip for package installation.. Discussion can be found at https://github.com/pypa/pip/issues/12330


Note: you may need to restart the kernel to use updated packages.


DEPRECATION: Loading egg at c:\users\domin\pycharmprojects\ggsn---magn\.venv\lib\site-packages\magn-1.0.0.0-py3.12.egg is deprecated. pip 24.3 will enforce this behaviour change. A possible replacement is to use pip for package installation.. Discussion can be found at https://github.com/pypa/pip/issues/12330


Note: you may need to restart the kernel to use updated packages.


DEPRECATION: Loading egg at c:\users\domin\pycharmprojects\ggsn---magn\.venv\lib\site-packages\magn-1.0.0.0-py3.12.egg is deprecated. pip 24.3 will enforce this behaviour change. A possible replacement is to use pip for package installation.. Discussion can be found at https://github.com/pypa/pip/issues/12330


Note: you may need to restart the kernel to use updated packages.


DEPRECATION: Loading egg at c:\users\domin\pycharmprojects\ggsn---magn\.venv\lib\site-packages\magn-1.0.0.0-py3.12.egg is deprecated. pip 24.3 will enforce this behaviour change. A possible replacement is to use pip for package installation.. Discussion can be found at https://github.com/pypa/pip/issues/12330


### Pobranie oraz Przygotowanie Danych

Aby przetestować, zbudowaną przez nas bibliotekę pobierzemy zbiór danych z platformy Kaggle.

Zbiór danych w formie bazy danych sqlite:
- Zbiór ocen piosenek muzycznych z platformy Pitchfork (Formalnie dalej nazywany `Pitchfork`) — [Link do danych z platformy Kaggle](https://www.kaggle.com/datasets/nolanbconaway/pitchfork-data)

In [2]:
from pathlib import Path
from typing import Final

resources_dir: Final[Path] = Path('resources/data/')

zip_data_file_pitchfork: Final[Path] = resources_dir.joinpath('pitchfork-data.zip')
database_path_pitchfork: Final[Path]  = resources_dir.joinpath('database.sqlite')

Pobranie zbioru danych i umieszczenie go w folderze `./resources/data/` do dalszej eksploatacji.

In [3]:
from os.path import exists
    
if not exists(zip_data_file_pitchfork):
    !kaggle datasets download -d nolanbconaway/pitchfork-data -p resources/data

Rozpakowanie danych do odpowiedniego katalogu: `./resources/data/`, aby móc użyć faktycznie bazę.

In [4]:
from zipfile import ZipFile

if not exists(database_path_pitchfork):
    with ZipFile(zip_data_file_pitchfork, 'r') as zip_ref:
        zip_ref.extractall(resources_dir)

Funkcje pomocnicze potrzebne do przekształcenia naszej pobranej bazy danych, ponieważ z niewiadomego mi powodu wszystkie bazy danych SQLite, pobrane z platformy kaggle nie zawierają np. kluczy głównych, obcych.

In [5]:
from typing import List, Tuple
import sqlite3

def get_table_columns_types(db_cursor: sqlite3.Cursor, table_name: str) -> List[Tuple[str, str]]:
    """Returns a list of tuples containing the column name and data type of given table in a database."""

    info = db_cursor.execute(f"""
        PRAGMA
            table_info({table_name});
    """).fetchall()

    return list((table_info[1], table_info[2]) for table_info in info)

def get_table_names(db_cursor: sqlite3.Cursor) -> List[str]:
    """Gets the names of all sqlite tables from a database."""
    db_cursor.execute("""
        SELECT
            name
        FROM
            sqlite_master
        WHERE
            type='table'
    """)

    return list(table[0] for table in db_cursor.fetchall())

def move_data_and_switch_tables(db_cursor: sqlite3.Cursor, table_name: str, temporary_table_name: str) -> None:
    """Moves data from a temporary table to the original table and switches the tables in a database."""

    db_cursor.executescript(f"""
        -- Move the data
        INSERT INTO
            {temporary_table_name}
        SELECT
            *
        FROM
            {table_name};
        
        -- Drop the old table
        DROP TABLE
            {table_name};
        
        -- Rename the new table
        ALTER TABLE
            {temporary_table_name}
        RENAME TO
            {table_name};
    """)

def create_table_with_primary_key(db_cursor: sqlite3.Cursor, table_name: str, temporary_table_name: str, column_name: str) -> None:
    """Creates a table with a primary key in a database."""

    table_columns = get_table_columns_types(db_cursor, table_name)

    db_cursor.execute(f"""
        CREATE TABLE {temporary_table_name} (
            {
                ', '.join(
                    f'{column} {data_type}'
                    if column != column_name
                    else f'{column} {data_type} PRIMARY KEY'
                    for column, data_type in table_columns
                )
            }
        );
    """)

def create_table_with_foreign_key(db_cursor: sqlite3.Cursor, table_name: str, column_name: str, foreign_table_name: str, foreign_column_name: str, temporary_foreign_table_name: str) -> None:
    """Creates a table with a foreign key in a database."""

    foreign_table_columns = get_table_columns_types(db_cursor, foreign_table_name)

    db_cursor.execute(f"""
        CREATE TABLE {temporary_foreign_table_name} (
            {
                ', '.join(
                    f'{column} {data_type}'
                    for column, data_type in foreign_table_columns
                )
            },
            CONSTRAINT fk_{foreign_table_name}_{foreign_column_name}
                FOREIGN KEY
                    ({foreign_column_name})
                REFERENCES
                    {table_name} ({column_name})
        );
    """)

Z uwagi, że całość musieliśmy zaimplementować w Python -ie to postanowiliśmy ograniczyć dane uczące do około 1000 rekordów z wszystkich tabel. Dla szybkości uczenia i dla ograniczenia złożoności pamięciowej użytkowników o trochę mniejszych komputerach.

Zapraszamy do bawienia się tymi parametrami do sprawdzenia wyników uczenia, predykcji przy zmniejszeniu lub zwiększeniu tej liczby. Cały zbiór danych ma około 18000 rekordów.

Wyczyścimy sobie też nasze dane, tak aby np. braki w jakiejś kolumnie, nie zepsuły nam wyników uczenia. Usuniemy też błędne i niepotrzebne duplikaty w kluczach głównych. Na spokojnie duplikaty wartości innych niż klucze zostawiamy w spokoju, ponieważ są one potrzebne dla ASA grafów w MAGN.

In [6]:
def limit_rows(db_cursor: sqlite3.Cursor, column_name: str, max_rows: int) -> None:
    """Limits the number of rows in a table in a database."""
    
    all_tables = get_table_names(db_cursor)
    
    # From every table and every column deletes rows that are NULL
    for table_name in all_tables:
        for given_column_name, _ in get_table_columns_types(db_cursor, table_name):
            db_cursor.execute(f"""
                DELETE FROM
                    {table_name}
                WHERE
                    {given_column_name} IS NULL;
            """)
    
    # From every table delete rows that are a duplicate of the column_name
    for table_name in all_tables:
        db_cursor.execute(f"""
            DELETE FROM
                {table_name}
            WHERE
                {column_name} IN (
                    SELECT
                        {column_name}
                    FROM
                        {table_name}
                    GROUP BY
                        {column_name}
                    HAVING
                        COUNT(*) > 1
                );
        """)
    
    # Find the intersection of a column_name in every table
    # Delete rows that are not in the intersection
    intersection: str = 'INTERSECT'.join(f"""
        SELECT
            {column_name}
        FROM
            {intersection_table_name}
    """ for intersection_table_name in all_tables)
    
    for table_name in all_tables:
        db_cursor.execute(f"""
            DELETE FROM
                {table_name}
            WHERE
                {column_name} NOT IN (
                    {intersection}
                );
        """)
    
    # Limit the number of rows in a table to max_rows
    for table_name in all_tables:
        db_cursor.execute(f"""
            DELETE FROM
                {table_name}
            WHERE
                {column_name} NOT IN (
                    SELECT
                        {column_name}
                    FROM
                        {table_name}
                    ORDER BY
                        {column_name}
                    LIMIT {max_rows}
                );
        """)

In [7]:
from typing import Final

MAX_ROWS: Final[int] = 1000

In [8]:
with sqlite3.connect(database_path_pitchfork) as conn:
    limit_rows(conn.cursor(), 'reviewid', MAX_ROWS)

Dodanie kluczy głównych i obcych do bazy danych, ponieważ nie zostały one dodane podczas tworzenia na platformie Kaggle, a jest to ważne podczas tworzenia MAGN.

W tym modelu przyjęliśmy, że klucze zawsze są jedno kolumnowe.

In [9]:
def add_foreign_key(db_cursor: sqlite3.Cursor, table_name: str, column_name: str, foreign_table_name: str, foreign_column_name: str) -> None:
    """Adds a foreign key to a given table column in a database."""

    temporary_foreign_table_name = f'{foreign_table_name}_'

    db_cursor.execute(f"""
        PRAGMA
            foreign_keys = OFF;
    """)

    create_table_with_foreign_key(db_cursor, table_name, column_name, foreign_table_name, foreign_column_name, temporary_foreign_table_name)
    move_data_and_switch_tables(db_cursor, foreign_table_name, temporary_foreign_table_name)

    db_cursor.execute(f"""
        PRAGMA
            foreign_keys = ON;
    """)

def add_primary_key(db_cursor: sqlite3.Cursor, table_name: str, column_name: str) -> None:
    """Adds a primary key to a given table column in a database."""

    temporary_table_name = f'{table_name}_'

    create_table_with_primary_key(db_cursor, table_name, temporary_table_name, column_name)
    move_data_and_switch_tables(db_cursor, table_name, temporary_table_name)

In [10]:
with sqlite3.connect(database_path_pitchfork) as conn:
    cursor = conn.cursor()

    cursor.execute(f"""BEGIN;""")
    try:
        add_primary_key(cursor, 'reviews', 'reviewid')

        add_foreign_key(cursor, 'reviews', 'reviewid', 'artists', 'reviewid')
        add_foreign_key(cursor, 'reviews', 'reviewid', 'content', 'reviewid')
        add_foreign_key(cursor, 'reviews', 'reviewid', 'genres', 'reviewid')
        add_foreign_key(cursor, 'reviews', 'reviewid', 'labels', 'reviewid')
        add_foreign_key(cursor, 'reviews', 'reviewid', 'years', 'reviewid')
    except Exception as error:
        conn.rollback()
        raise error
    else:
        conn.commit()

### Pełny ERD dla bazy danych Pitchfork

Poniżej przedstawiam schemat bazy danych `Pitchfork`.

![ERD Diagram for the database schema](docs/images/database_erd_pitchfork.png)

Został wygenerowany przez nas przy pomocy PyCharm oraz znajduje się w folderze `docs/images/database_erd_pitchfork.png`

Zobaczmy, jak nasze funkcje od konwersji do MAGN widzą powyższa baza danych.

Warto zauważyć, że meta-tabela (`sqlite_master`) nie jest brana pod uwagę, ponieważ z technicznego punktu widzenia to nie tabela, tylko interface do metadanych bazy.

In [11]:
from magn.database.sqlite3 import get_table_names

all_tables_pitchfork = get_table_names(database_path_pitchfork)

all_tables_pitchfork

['reviews', 'artists', 'content', 'genres', 'labels', 'years']

Teraz zobaczmy, czy nasze funkcje wyłuskały wszystkie zdefiniowane wcześniej relacje. Dla przypomnienia dla poszczególnych tabel to powinno tak wyglądać:
1. `Pitchfork`:
   - Klucz główny w `reviews`, `reviewid`
   - Klucz obcy w `artists`, `reviewid`
   - Klucz obcy w `content`, `reviewid`
   - Klucz obcy w `genres`, `reviewid`
   - Klucz obcy w `labels`, `reviewid`
   - Klucz obcy w `years`, `reviewid`

Poniżej zamieszczam jak należy rozumieć to wyjście zmiennej (`keys_%`):

![ERD Diagram with key output explanation](docs/images/database_erd_pitchfork_foreign_keys.png)

Oznaczenia:
- `Czerwony` - Obecna tabela, której klucze obce się tyczą.
- `Niebieski` - Tabela końcowa zawierająca klucz główny, który referenconujemy.
- `Żółty` - Kolumna z kluczem obcym.
- `Zielony` - Kolumna z kluczem głównym.

In [12]:
from magn.database.sqlite3 import SQLite3KeysReader

keys_reader = SQLite3KeysReader(database_path_pitchfork, all_tables_pitchfork)
keys_pitchfork = keys_reader.read()

keys_pitchfork

{'reviews': Keys(primary_keys=['reviewid'], foreign_keys={}),
 'artists': Keys(primary_keys=[], foreign_keys={'reviews': ('reviewid', 'reviewid')}),
 'content': Keys(primary_keys=[], foreign_keys={'reviews': ('reviewid', 'reviewid')}),
 'genres': Keys(primary_keys=[], foreign_keys={'reviews': ('reviewid', 'reviewid')}),
 'labels': Keys(primary_keys=[], foreign_keys={'reviews': ('reviewid', 'reviewid')}),
 'years': Keys(primary_keys=[], foreign_keys={'reviews': ('reviewid', 'reviewid')})}

Tutaj zwrócimy jakie dane z tabel w postaci pandas Dataframe -u zostały znalezione (przynajmniej skrótowo)

In [13]:
from magn.database.sqlite3 import SQLite3DataReader

data_reader = SQLite3DataReader(database_path_pitchfork, all_tables_pitchfork, keys_pitchfork)
data_reader.read()

{'reviews':                                   title           artist  \
 reviewid                                                   
 1                         young forever        aberfeldy   
 6                  pure tone audiometry         aarktica   
 11        homesick and happy to be here         aberdeen   
 31                          rolled gold           action   
 33             the scene's out of sight     actionslacks   
 ...                                 ...              ...   
 5329                 advisory committee            mirah   
 5334                radio-free brooklyn       pete miser   
 5339                          bangzilla  mix master mike   
 5341                           three ep      mobius band   
 5342                 city vs country ep      mobius band   
 
                                                         url  score  \
 reviewid                                                             
 1         http://pitchfork.com/reviews/albums/1-you

Tak by wyglądało odczytanie przez nas gotowej bazy danych w całości.

In [14]:
from magn.database.database import Database

database_pitchfork = Database.from_sqlite3(database_path_pitchfork)

database_pitchfork

Database(all_data={'reviews': Table(data=                                  title           artist  \
reviewid                                                   
1                         young forever        aberfeldy   
6                  pure tone audiometry         aarktica   
11        homesick and happy to be here         aberdeen   
31                          rolled gold           action   
33             the scene's out of sight     actionslacks   
...                                 ...              ...   
5329                 advisory committee            mirah   
5334                radio-free brooklyn       pete miser   
5339                          bangzilla  mix master mike   
5341                           three ep      mobius band   
5342                 city vs country ep      mobius band   

                                                        url  score  \
reviewid                                                             
1         http://pitchfork.com/reviews

### Wnioski:
- ...

### Co się nauczyliśmy:
- Jak działają poszczególne struktury danych. Jakie mają operacje wstawianie / usuwania itd.:
    - ASA
    - MAGN
    - B-Drzewa
- Jak przeprowadzić uczenie / predykcję dla MAGN.
- Jak zrobić transformatę arbitralnej bazy danych do MAGN.
- Jak pracować z bazą danych typu SQLite przy pomocy Pythona:
    - Jak utworzyć klucze główne, obce dla tabel, już wcześniej stworzonych.
    - Jak dostać jakie kolumny, tabele ma baza danych (meta dane o schemacie bazy).
- Wizualizacje przy pomocy Graphviza MAGN (Jak działa Graphviz i jak go użyć do wizualizacji).
- Jak w PyCharm wygenerować diagram ERD dla bazy danych.
- Jak działa i jak zaimplementować sortowanie topologiczne.
- Jak poprawnie stworzyć package w python- ie.

### Źródła:

Wykorzystane przez nas materiały / artykuły niezbędne do stworzenia tego projektu.
- Construction and Training of Multi-Associative Graph Networks, Adrian Horzyk, Daniel Bulanda, and Janusz A. Starzyk
- Associative Data Structures and Associative Neural Graphs, Adrian Horzyk [LINK](https://home.agh.edu.pl/~horzyk/lectures/ci/CI-AssociativeNeuralGraphs.pdf)
- ASA-graphs for efficient data representation and processing [LINK](https://sciendo.com/article/10.34768/amcs-2020-0053)
- How to Use Associative Knowledge Graphs to Build Efficient Knowledge Models [LINK](https://grapeup.com/blog/associative-knowledge-graphs)
- Building ASA Graphs [LINK](https://youtube.com/watch?v=K1a2Bk8NrYQ&si=n5OPz9_Em5TVgX_2)